**Using scrapy to pull articles and text from better homes and gardens website from the gardening/houseplants section.**

In [1]:
import requests     # how python goes onto the internet!
import re           # regex
from bs4 import BeautifulSoup # a python HTML parser (version 3)

In [2]:
# Let's grab the raw html from the page
r = requests.get('https://www.bhg.com/gardening/houseplants/')

# create a beautifulsoup object
b = BeautifulSoup(r.text, 'html.parser') 

In [3]:
b.prettify() # will print the html nicely

'<!DOCTYPE html>\n<head>\n <meta charset="utf-8"/>\n <title data-socialdescription="" data-socialtitle="">\n  Houseplants\n </title>\n <meta content="width=device-width, initial-scale=1.0" name="viewport"/>\n <style>\n  meta.foundation-version{font-family:"/5.5.1/"}meta.foundation-mq-small{font-family:"/only screen/";width:0}meta.foundation-mq-small-only{font-family:"/only screen and (max-width: 42.375rem)/";width:0}meta.foundation-mq-medium{font-family:"/only screen and (min-width:42.4375rem)/";width:42.4375rem}meta.foundation-mq-medium-only{font-family:"/only screen and (min-width:42.4375rem) and (max-width:62.8125rem)/";width:42.4375rem}meta.foundation-mq-large{font-family:"/only screen and (min-width:62.875rem)/";width:62.875rem}meta.foundation-mq-large-only{font-family:"/only screen and (min-width:62.875rem) and (max-width:80.625rem)/";width:62.875rem}meta.foundation-mq-xlarge{font-family:"/only screen and (min-width:80.6875rem)/";width:80.6875rem}meta.foundation-mq-xlarge-only{fo

In [6]:
# find all span tags

b.find_all('a', href=True)

[<a class="navLabel" href="#" id="superNavLink">
 <div class="hamburger">
 <span class="line line-t"></span>
 <span class="line line-m"></span>
 <span class="line line-b"></span>
 </div>
 </a>,
 <a href="https://secure.bhg.com/common/profile/regStep1.jsp?regSource=1000&amp;returnURL=/" id="navJoinNow" title="Join Now!"><span>Join Now</span></a>,
 <a data-reglink="true" href="https://secure.bhg.com/common/profile/login1.jsp?regSource=9381&amp;returnURL=/" id="navLogIn" title="Log in"><span>Log In</span></a>,
 <a class="hamburgerProfileLink--registered" href="https://secure.bhg.com/my/profile/">Hi, Friend</a>,
 <a href="#"><span>Subscribe</span></a>,
 <a class="topMenuItem" href="https://www.bhg.com/recipes/"><span>Recipes</span></a>,
 <a class="topMenuItem" href="https://www.bhg.com/decorating/"><span>Decorating</span></a>,
 <a class="topMenuItem" href="https://www.bhg.com/home-improvement/"><span>Home Improvement</span></a>,
 <a class="topMenuItem" href="https://www.bhg.com/gardening/"

In [7]:
for link in b.findAll("a", attrs={'itemprop':'url'})[3:]:
        print(link.string)

Indoor Plants for Low Light
                        
24 Beautiful Blooming Houseplants
                        
10 Photos That Prove Kokedama is the Best Plant Trend Right Now
                        
The Easiest Houseplants You Can Grow
                        
Top Fragrant Houseplants
                        
How to Make a Pallet Planter
                        
Must-Know Tips for Watering Succulents
                        
5 Stunning Air Plant Ideas
                        
Ficus
                        
Philodendron
                        
Snake Plant
                        
10 Fresh Ways to Add Succulents to Your Home
                        
Houseplants You Can't Kill
                        
9 Top Ferns to Grow as Houseplants
                        
Top Terrarium Plants
                        
Terrarium Fairy Garden Inspiration
                        
3-Step Blooming Spring Centerpieces
                        
Top 10 Succulent Plants for the Home
                        


In [1]:
# Importing in each cell because of the kernel restarts.
import scrapy
from scrapy.crawler import CrawlerProcess


class BHGSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "BHG"
    
    # URL(s) to start with.
    start_urls = [
        'https://www.bhg.com/gardening/houseplants/',
    ]

    # Use XPath to parse the response we get.
    def parse(self, response):
        
        # Iterate over every <article> element on the page.
        for div in response.xpath('/div'):
            
            # Yield a dictionary with the values we want.
            yield {
                # This is the code to choose what we want to extract
                # You can modify this with other Xpath expressions to extract other information from the site
                'data': div.xpath('div[@class="item fromTigercub slideshow processed').extract_first(),
                #'data_content_id': div.xpath('div[@class="item fromTigercub slideshow processed').extract_first(),
                'title': div.xpath('div/h3/a[@class="title"]/text()').extract_first(),
                'description': div.xpath('div/p[@class="description"]/p/text()').extract(),
                #'tags': article.xpath('*/span[@class="name"]/a/text()').extract()
            }

# Tell the script how to run the crawler by passing in settings.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'houseplants.json',  # Name our storage file.
    'LOG_ENABLED': False           # Turn off logging for now.
})

# Start the crawler with our spider.
process.crawl(BHGSpider)
process.start()
print('Success!')

Success!


In [3]:
import pandas as pd

houseplants = pd.read_json('houseplants.json', orient='records')
print(houseplants.shape)
print(houseplants.head())

ValueError: Expected object or value

In [1]:
# Importing in each cell because of the kernel restarts.
import scrapy
import re
from scrapy.crawler import CrawlerProcess

class ESSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "ESS"
    
    # URL(s) to start with.
    start_urls = [
        'http://www.everydaysexism.com',
    ]

    # Use XPath to parse the response we get.
    def parse(self, response):
        
        # Iterate over every <article> element on the page.
        for article in response.xpath('//article'):
            
            # Yield a dictionary with the values we want.
            yield {
                'name': article.xpath('header/h2/a/@title').extract_first(),
                'date': article.xpath('header/section/span[@class="entry-date"]/text()').extract_first(),
                'text': article.xpath('section[@class="entry-content"]/p/text()').extract(),
                'tags': article.xpath('*/span[@class="tag-links"]/a/text()').extract()
            }
        # Get the URL of the previous page.
        next_page = response.xpath('//div[@class="nav-previous"]/a/@href').extract_first()
        
        # There are a LOT of pages here.  For our example, we'll just scrape the first 9.
        # This finds the page number. The next segment of code prevents us from going beyond page 9.
        pagenum = int(re.findall(r'\d+',next_page)[0])
        
        # Recursively call the spider to run on the next page, if it exists.
        if next_page is not None and pagenum < 10:
            next_page = response.urljoin(next_page)
            # Request the next page and recursively parse it the same way we did above
            yield scrapy.Request(next_page, callback=self.parse)

# Tell the script how to run the crawler by passing in settings.
# The new settings have to do with scraping etiquette.          
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'data.json',       # Name our storage file.
    'LOG_ENABLED': False,          # Turn off logging for now.
    'ROBOTSTXT_OBEY': True,
    'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True
})

# Start the crawler with our spider.
process.crawl(ESSpider)
process.start()
print('Success!')

Success!


In [2]:
import pandas as pd

# Checking whether we got data from all 9 pages
ESSdf=pd.read_json('data.json', orient='records')
print(ESSdf.shape)
print(ESSdf.head())

(90, 4)
        date           name  \
0 2018-07-16        Michael   
1 2018-07-16       Alkistis   
2 2018-07-16  Rizaye Rodela   
3 2018-07-16             15   
4 2018-07-16            Sue   

                                                tags  \
0  [Home, Public space, Public Transport, School,...   
1                                           [School]   
2                                             [Home]   
3                                             [Home]   
4                                             [Home]   

                                                text  
0  [Hello., \nI am a 28 year old man.  I am just ...  
1  [A few weeks ago I had a conversation with my ...  
2  [I’m from a small district dinajpur from a sma...  
3  [3 years ago, when I was around 12 I believe, ...  
4  [On leaving my house yesterday, with my husban...  
